### Contextualizing Transcriptomic Data

In [13]:
#!/usr/bin/python

# Dependencies
import copy
import time
import numpy
import cobra
import pandas
import bisect
import symengine
from cobra.util import solver
from optlang.symbolics import Zero
from cobra.manipulation.delete import remove_genes
from cobra.flux_analysis.sampling import ACHRSampler
from cobra.flux_analysis import flux_variability_analysis

# Read in transcriptomic read abundances, default is tsv with no header 
def read_transcription_file(read_abundances_file, header=False, replicates=False, sep='\t'):
    '''Generates dictionary of transcriptomic abundances from a file.
    
    Parameters
    ----------
    read_abundances_file : string
        User-provided file name which contains gene IDs and associated transcription values
    header : boolean
        Defines if read abundance file has a header that needs to be ignored
        default is no header
    replicates : boolean
        Defines if read abundances contains replicates and medians require calculation
        default is no replicates
    sep: string
        Defines what character separates entries on each line
        defaults to tab (.tsv)
    '''
    abund_dict = {}
    with open(read_abundances_file, 'r') as transcription:
        if header == True: header_line = transcription.readline()

        for line in transcription:
            line = line.split(sep)
            gene = str(line[0])
            
            if replicates == True:
                abundance = float(numpy.median([float(x) for x in line[1:]]))
            else:
                abundance = float(line[1])
            
            if gene in abund_dict.keys():
                abund_dict[gene] += abundance
            else:
                abund_dict[gene] = abundance

    return abund_dict

        
# Ensure that the user provided model and transcriptomic data are ready for RIPTiDe
def initialize_model(model):
    
    # Create a copy of the original model and set new id
    riptide_model = copy.deepcopy(model)
    riptide_model.id = str(riptide_model.id) + '_riptide'
    
    # Check that the model can grow
    solution = riptide_model.optimize()
    if solution.objective_value < 1e-6 or str(solution.objective_value) == 'nan':
        raise ValueError('ERROR: Provided model objective cannot carry flux! Please correct')
    
    # Calculate flux ranges and remove totally blocked reactions
    flux_span = flux_variability_analysis(riptide_model, fraction_of_optimum=0.01)
    flux_ranges = {}
    blocked_rxns = []
    for rxn_id, min_max in flux_span.iterrows():
        if max(abs(min_max)) < 1e-6:
            blocked_rxns.append(rxn_id)
        else:
            flux_ranges[rxn_id] = [min(min_max), max(min_max)]
    for rxn in blocked_rxns: 
        riptide_model.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)

    return riptide_model


# Converts a dictionary of transcript distribution percentiles
def assign_coefficients(raw_transcription_dict, model, percentiles, min_coefficients):
    
    # Screen transcriptomic abundances for genes that are included in model
    transcription_dict = {}
    for gene in model.genes:
        try:
            transcription_dict[gene.id] = raw_transcription_dict[gene.id]
        except KeyError:
            continue
    
    # Calculate transcript abundance cutoffs
    distribution = transcription_dict.values()
    abund_cutoffs = [numpy.percentile(distribution, x) for x in percentiles]
    
    # Screen transcript distribution by newly defined abundance intervals
    coefficient_dict = {}
    for gene in transcription_dict.iterkeys():
        transcription = transcription_dict[gene]
        if transcription in abund_cutoffs:
            index = abund_cutoffs.index(transcription)
            min_coefficient = min_coefficients[index]
        else:
            index = bisect.bisect_right(abund_cutoffs, transcription) - 1
            min_coefficient = min_coefficients[index]
                    
        # Assign corresponding coefficients to reactions associated with each gene
        for rxn in list(model.genes.get_by_any(gene)[0].reactions):            
            if rxn.id in coefficient_dict.keys():
                coefficient_dict[rxn.id].append(min_coefficient)
            else:
                coefficient_dict[rxn.id] = [min_coefficient]
    
    # Assign final coefficients
    nogene_coefficient = numpy.median(min_coefficients)
    for rxn in model.reactions:
        try:
            # Take smallest value for reactions assigned multiple coefficients
            coefficient_dict[rxn.id] = min(coefficient_dict[rxn.id])
        except KeyError:
            coefficient_dict[rxn.id] = nogene_coefficient
            continue
    
    return coefficient_dict


# Read in user defined reactions to keep or exclude
def incorporate_user_defined_reactions(rm_rxns, reaction_file):
    
    print('Integrating user definitions...')
    sep = ',' if '.csv' in str(reaction_file) else '\t'
    
    # Check if file actually exists    
    try:
        with open(reaction_file, 'r') as reactions:
            include_rxns = set(reactions.readline().split(sep))
            exclude_rxns = set(reactions.readline().split(sep))
    except FileNotFoundError:
        raise FileNotFoundError('ERROR: Defined reactions file not found! Please correct.')
        
    rm_rxns = rm_rxns.difference(include_rxns)
    rm_rxns |= exclude_rxns

    return rm_rxns


# Determine those reactions that carry flux in a pFBA objective set to a threshold of maximum
def constrain_and_analyze_model(model, coefficient_dict, fraction, sampling_depth):
    
    with model as constrained_model:

        # Apply weigths to new expression
        pfba_expr = Zero
        if sampling_depth == 'minimization':
            for rxn in constrained_model.reactions:
                pfba_expr += coefficient_dict[rxn.id] * rxn.forward_variable
                pfba_expr += coefficient_dict[rxn.id] * rxn.reverse_variable
        else:
            coeff_range = float(max(coefficient_dict.values())) + float(min(coefficient_dict.values()))
            for rxn in constrained_model.reactions:
                max_coeff = coeff_range - float(coefficient_dict[rxn.id])
                pfba_expr += max_coeff * rxn.forward_variable
                pfba_expr += max_coeff * rxn.reverse_variable
                
        # Calculate sum of fluxes constraint
        if sampling_depth == 'minimization':
            prev_obj_val = constrained_model.slim_optimize()
            # Set previous objective as a constraint, allow deviation
            prev_obj_constraint = constrained_model.problem.Constraint(constrained_model.objective.expression, lb=prev_obj_val*fraction, ub=prev_obj_val)
            constrained_model.add_cons_vars([prev_obj_constraint])
            constrained_model.objective = constrained_model.problem.Objective(pfba_expr, direction='min', sloppy=True)
            constrained_model.solver.update()
            solution = constrained_model.optimize()
            
            # Determine reactions that do not carry any flux in the constrained model
            inactive_rxns = set([rxn.id for rxn in constrained_model.reactions if abs(solution.fluxes[rxn.id]) < 1e-6])
            return inactive_rxns
        
        else:        
            # Explore solution space of constrained model with flux sampling, allow deviation
            constrained_model.objective = constrained_model.problem.Objective(pfba_expr, direction='max', sloppy=True)
            solution = constrained_model.optimize()
            flux_sum_obj_val = solution.objective_value
            flux_sum_constraint = constrained_model.problem.Constraint(pfba_expr, lb=flux_sum_obj_val*fraction, ub=flux_sum_obj_val)
            constrained_model.add_cons_vars([flux_sum_constraint])
            constrained_model.solver.update()
            
            # Perform flux sampling (or FVA)
            flux_object = explore_flux_ranges(constrained_model, sampling_depth, fraction)
            return flux_object
    

# Prune model based on blocked reactions from minimization as well as user-defined reactions
def prune_model(new_model, rm_rxns, defined_rxns, conserve):
      
    # Integrate user definitions
    if defined_rxns != False: 
        rm_rxns = incorporate_user_defined_reactions(rm_rxns, defined_rxns)
        
    # Parse elements highlighted for pruning based on GPRs
    if conserve == 'y':
        final_rm_rxns = []
        for rxn in rm_rxns:
            test = 'pass'
            current_genes = list(new_model.reactions.get_by_id(rxn).genes)
            for gene in current_genes:
                for rxn_sub in gene.reactions:
                    if rxn_sub.id not in rm_rxns:
                        test = 'fail'
                    else:
                        pass
            
        	if test == 'pass': final_rm_rxns.append(rxn)
    else:
    	final_rm_rxns = rm_rxns
                        
    # Screen for duplicates
    final_rm_rxns = list(set(final_rm_rxns))
    
    # Prune inactive reactions
    for rxn in final_rm_rxns:
        new_model.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)
    
    # Prune possible residual orphans, kind of sloppy but it's the only way 
    # I've found for it to actually thoroughly remove orphans
    removed = 1
    while removed == 1:
        removed = 0
        for cpd in new_model.metabolites:
            if len(cpd.reactions) == 0:
                cpd.remove_from_model(remove_orphans=True); removed = 1
        for rxn in new_model.reactions:
            if len(rxn.metabolites) == 0: 
                rxn.remove_from_model(remove_orphans=True); removed = 1
    
    return new_model


# Analyze the possible ranges of flux in the constrained model
def explore_flux_ranges(model, samples, fraction):
    
    try:
        sampling_object = ACHRSampler(model)
        flux_object = sampling_object.sample(samples)        
        analysis = 'flux_sampling'
    except:
        # Handle errors for models that are now too small
        print('Constrained solution space too narrow for sampling, performing FVA instead')        
        flux_object = flux_variability_analysis(model, fraction_of_optimum=fraction)
        analysis = 'fva'
        
    return flux_object, analysis
    
    
# Calculate approximate volume of solution space, treated as ellipsoid
def calculate_polytope_volume(model, fraction):
    
    flux_span = flux_variability_analysis(model, fraction_of_optimum=fraction)
    bounds = {}
    for rxn_id, min_max in flux_span.iterrows(): bounds[rxn_id] = [min(min_max), max(min_max)]

    # Compile a list of radii from flux ranges
    radii = []
    for rxn in bounds.iterkeys():
        if bounds[rxn] == [0.0, 0.0]:
            continue
        else:
            diameter = abs(bounds[rxn][0]) + abs(bounds[rxn][1])
            radii.append(numpy.median(diameter) / 2.0)
    
    # Calculate volume 
    volume = (4.0/3.0) * numpy.pi * max(radii) * numpy.median(radii) * min(radii)
    volume = round(volume, 3)
    
    return volume


# Reports how long RIPTiDe took to run
def operation_report(start_time, model, riptide, old_vol, new_vol):
    
    # Pruning
    perc_removal = 100.0 - ((float(len(riptide.reactions)) / float(len(model.reactions))) * 100.0)
    perc_removal = round(perc_removal, 1)
    print('\nReactions pruned to ' + str(len(riptide.reactions)) + ' from ' + str(len(model.reactions)) + ' (' + str(perc_removal) + '% reduction)')
    perc_removal = 100.0 - ((float(len(riptide.metabolites)) / float(len(model.metabolites))) * 100.0)
    perc_removal = round(perc_removal, 1)
    print('Metabolites pruned to ' + str(len(riptide.metabolites)) + ' from ' + str(len(model.metabolites)) + ' (' + str(perc_removal) + '% reduction)')
    
    # Flux through objective
    new_ov = round(riptide.slim_optimize(), 3)
    old_ov = round(model.slim_optimize(), 3)
    per_shift = 100.0 - ((new_ov / old_ov) * 100.0)
    if per_shift == 0.0:
        pass
    elif per_shift > 0.0:
        per_shift = round(abs(per_shift), 2)
        print('\nFlux through the objective REDUCED to ' + str(new_ov) + ' from ' + str(old_ov) + ' (' + str(per_shift) + '% shift)')
    elif per_shift < 0.0:
        per_shift = round(abs(per_shift), 2)
        print('\nFlux through the objective INCREASED to ' + str(new_ov) + ' from ' + str(old_ov) + ' (' + str(per_shift) + '% shift)')
    
    # Solution space volume
    vol_shift = 100.0 - ((new_vol / old_vol) * 100.0)
    if new_vol > 100000 or old_vol > 100000:
        pass
    elif vol_shift < 0.0:
        vol_shift = round(abs(vol_shift), 2)
        print('Solution space ellipsoid volume INCREASED to ~' + str(new_vol) + ' from ~' + str(old_vol) + ' (' + str(vol_shift) + '% shift)')
    elif vol_shift > 0.0:
        vol_shift = round(vol_shift, 2)
        print('Solution space ellipsoid volume DECREASED to ~' + str(new_vol) + ' from ~' + str(old_vol) + ' (' + str(vol_shift) + '% shift)')
    else:
        print('No change in Solution space volume')
    
    # Check that prune model can still achieve flux through the objective (just in case)
    if riptide.slim_optimize() < 1e-6 or str(riptide.slim_optimize()) == 'nan':
        print('\nWARNING: Contextualized model objective can no longer carry flux')
    
    # Run time
    duration = time.time() - start_time
    if duration < 60.0:
        duration = round(duration)
        print '\nRIPTiDe completed in ' + str(duration) + ' seconds\n'
    elif duration < 3600.0:
        duration = round((duration / 60.0), 1)
        print '\nRIPTiDe completed in ' + str(duration) + ' minutes\n'
    else:
        duration = round((duration / 3600.0), 1)
        print '\nRIPTiDe completed in ' + str(duration) + ' hours\n'


# Create context-specific model based on transcript distribution
def riptide(model, transcription, defined = False, sampling = 10000, percentiles = [50.0, 62.5, 75.0, 87.5], coefficients = [1.0, 0.5, 0.1, 0.01, 0.001], fraction = 0.8, conservative = 'n'):
    '''Reaction Inclusion by Parsimony and Transcriptomic Distribution or RIPTiDe
    
    Creates a contextualized metabolic model based on parsimonious usage of reactions defined
    by their associated transcriptomic abundances. Returns a pruned, context-specific cobra.Model 
    and a pandas.DataFrame of associated flux sampling distributions

    Parameters
    ----------
    model : cobra.Model
        The model to be contextualized
    transcription : dictionary
        Dictionary of transcript abundances, output of read_transcription_file()
    defined : False or File
        Text file containing reactions IDs for forced inclusion listed on the first line and exclusion 
        listed on the second line (both .csv and .tsv formats supported)
    sampling : int or False
        Number of flux samples to collect, default is 10000, If False, sampling skipped
    percentiles : list of floats
        Percentile cutoffs of transcript abundance for linear coefficient assignments to associated reactions
        Default is [50.0, 62.5, 75.0, 87.5]
    coefficients : list of floats
        Linear coefficients to weight reactions based on distribution placement
        Default is [1.0, 0.5, 0.1, 0.01, 0.001]
    fraction : float
        Minimum percent of optimal objective value during FBA steps
        Default is 0.8
    conservative : str
        Conservatively remove inactive reactions based on genes
        Either 'y' or 'n', default in 'n' (no)
    '''

    start_time = time.time()
    
    # Correct some possible user error
    if sampling == False:
        pass
    elif sampling <= 0: 
        sampling = 10000
    else: 
        samples = int(sampling)
    if len(set(transcription.values())) == 1:
        raise ValueError('ERROR: All transcriptomic abundances are identical! Please correct')
    if len(coefficients) != len(percentiles) + 1:
        raise ValueError('ERROR: Invalid ratio of percentile cutoffs to linear coefficients! Please correct')
    fraction = float(fraction)
    if fraction <= 0.0:
        fraction = 0.8
    percentiles.sort() # sort ascending
    coefficients.sort(reverse=True) # sort descending
    if conservative not in ['y','n']: conservative = 'y'

    # Check original model functionality
    # Partition reactions based on transcription percentile intervals, assign corresponding reaction coefficients
    print('\nInitializing model and parsing transcriptome...')
    riptide_model = initialize_model(model)
    orig_volume = calculate_polytope_volume(riptide_model, fraction)
    coefficient_dict = assign_coefficients(transcription, riptide_model, percentiles, coefficients)
    
    # Prune now inactive network sections based on coefficients
    print('Pruning zero flux subnetworks...')
    rm_rxns = constrain_and_analyze_model(riptide_model, coefficient_dict, fraction, 'minimization')
    riptide_model = prune_model(riptide_model, rm_rxns, defined, conservative)
    new_volume = calculate_polytope_volume(riptide_model, fraction)

    # Find optimal solution space based on transcription and final constraints
    if sampling != False:
        print('Sampling context-specific solution space (longest step)...')
        flux_object, analysis_type = constrain_and_analyze_model(riptide_model, coefficient_dict, fraction, samples)
        operation_report(start_time, model, riptide_model, orig_volume, new_volume)
        return riptide_model, flux_object
    
    else:
        operation_report(start_time, model, riptide_model, orig_volume, new_volume)
        return riptide_model


### Testing with Toy Model

In [3]:
import numpy
import pandas
import operator
from cobra.flux_analysis.parsimonious import *

# Use FBA calculations to find new shadow prices
def shadow_prices(model, compartment='all', top=25):
    with model as m: solution = pfba(m)
    
    shadow_prices = {}
    for cpd, price in solution.shadow_prices.iteritems():
        cpd = model.metabolites.get_by_any(cpd)[0]
        if compartment != 'all' and compartment != cpd.compartment:
            continue
        else:
            if price > 0.0: shadow_prices[cpd.name] = price

    sorted_prices = sorted(shadow_prices.items(), key=operator.itemgetter(1), reverse=True)[0:top]
    sorted_prices = pandas.DataFrame(sorted_prices, columns=['metabolite', 'shadow_price'])
    
    return sorted_prices


In [4]:
# Load in example model
toy_model = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/example_GENRE.sbml')

# gene1 = Glucose transporter
# gene2 = Proline transporter
# gene3 = Glycine transporter
# gene4 = Hydrogen efflux
# gene5 = Carbon dioxide efflux
# gene6 = Phosphate transporter
# gene7 = Glycolysis
# gene8 = Stickland fermentation

In [4]:
toy_model

Name,iTEST
Memory address,0x07f53912035d0
Number of metabolites,14
Number of reactions,16
Objective expression,0.0 + 1.0*DM_atp_c - 1.0*DM_atp_c_reverse_1b037
Compartments,"c, e"


In [5]:
shadow_prices(toy_model)

,metabolite,shadow_price
0,ATP,8.0
1,CO2,2.0
2,Glycine,2.0
3,Pi,2.0
4,Proline,2.0
5,Glucose,2.0
6,ADP,1.0


In [6]:
# Find most parsimonious route of flux
from cobra.flux_analysis.parsimonious import pfba
toy_solution = pfba(toy_model)
print(toy_solution.fluxes)

EX_gluc_e   -1000.0
rxn1         1000.0
EX_pro_e        0.0
rxn2            0.0
EX_gly_e        0.0
rxn3            0.0
EX_h_e       1000.0
rxn4         1000.0
EX_co2_e        0.0
rxn5            0.0
EX_pi_e     -1000.0
rxn6         1000.0
EX_adp_c    -1000.0
rxn7         1000.0
rxn8            0.0
DM_atp_c     1000.0
Name: fluxes, dtype: float64


In [5]:
# Create associated transcriptomes
glucose_transcriptome = {'gene1':100, 'gene2':1, 'gene3':1, 'gene4':1, 
                         'gene5':1, 'gene6':100, 'gene7':10000, 'gene8':1}
peptide_transcriptome = {'gene1':1, 'gene2':100, 'gene3':100, 'gene4':1, 
                         'gene5':1, 'gene6':1, 'gene7':1, 'gene8':10000}

In [15]:
# Contextualize toy model
toy_model_glucose, glucose_samples = riptide(toy_model, glucose_transcriptome)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...

Reactions pruned to 9 from 16 (43.8% reduction)
Metabolites pruned to 8 from 14 (42.9% reduction)

RIPTiDe completed in 1.0 seconds



In [9]:
shadow_prices(toy_model_glucose)

,metabolite,shadow_price
0,ATP,8.0
1,Pi,2.0
2,Glucose,2.0
3,ADP,1.0


In [11]:
# Contextualize toy model
toy_model_peptide, peptide_samples = riptide(toy_model, peptide_transcriptome)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 13 from 16 (18.8% reduction)
Metabolites pruned to 12 from 14 (14.3% reduction)

No change in flux through the objective

RIPTiDe completed in 1.6 minutes



In [12]:
shadow_prices(toy_model_peptide)

,metabolite,shadow_price
0,ATP,12.0
1,Pi,2.0
2,Proline,2.0
3,Glycine,2.0
4,ADP,1.0


In [10]:
# Test difference in objective fluxes
gluc_arp = glucose_samples['DM_atp_c']
pep_arp = peptide_samples['DM_atp_c']

import scipy.stats
scipy.stats.wilcoxon(x=gluc_arp, y=pep_arp)

WilcoxonResult(statistic=11743036.0, pvalue=0.0)

### Testing with E.coli K-12 MG1655 model

In [17]:
iJO1366 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/iJO1366.xml')
iJO1366.objective = iJO1366.reactions.BIOMASS_Ec_iJO1366_WT_53p95M

# Open all exchanges
exchanges = set()
for rxn in iJO1366.reactions:
    if len(rxn.reactants) == 0 or len(rxn.products) == 0:
        rxn.bounds = (min(rxn.lower_bound, -1000), max(rxn.upper_bound, 1000))
        exchanges |= set([rxn.id])

In [2]:
def max_doubling_time(model):
    
    with model as m: 
        growth =  m.slim_optimize()
    
    if growth < 1e-6:
        growth = 'No growth'
    else:    
        growth = (1.0 / growth) * 3600.0
        if growth < 60.0:
            growth = str(round(growth, 1)) + ' minutes'
        else:
            growth = growth / 60.0
            growth = str(round(growth, 3)) + ' hours'
            
    print(growth)


def collect_doubling_times(flux_samples, biomass):
    biomass = list(flux_samples[biomass])
    times = []
    
    for x in biomass:
        growth = (1.0 / x) * 3600.0 # Calculated in minutes
        growth = round(growth, 2)
        times.append(growth)
        
    return times


def collect_growth_rates(flux_samples, biomass):
    biomass = list(flux_samples[biomass])
    rates = []
    
    for x in biomass:
        rate = x / 60.0
        rate = round(rate, 3)
        rates.append(rate)
        
    return rates

In [7]:
max_doubling_time(iJO1366)

34.0 minutes


In [18]:
iJO1366

Name,iJO1366
Memory address,0x07f8f897aa3d0
Number of metabolites,1805
Number of reactions,2583
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


### Flux sampling on base model

In [8]:
# Prune blocked reactions
flux_span = flux_variability_analysis(iJO1366, fraction_of_optimum=0.01)
blocked_rxns = []
for rxn_id, min_max in flux_span.iterrows():
    if max(abs(min_max)) < 1e-6: blocked_rxns.append(rxn_id)
for rxn in blocked_rxns: 
    iJO1366.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)

In [11]:
# Constrain possible solutions
obj_val = iJO1366.slim_optimize()
obj_constraint = iJO1366.problem.Constraint(iJO1366.objective.expression, lb=obj_val*0.8, ub=obj_val)
iJO1366.add_cons_vars([obj_constraint])
iJO1366.solver.update()

In [13]:
# Flux sampling of base model
iJO1366_sampling_object = ACHRSampler(iJO1366)
iJO1366_flux_samples = iJO1366_sampling_object.sample(5000)

[-69727.61, 92.4, 665273.68]


In [40]:
# Collect base model growth information
base_rates = collect_growth_rates(iJO1366_flux_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')

# Screen data against negative values
screened_base_rates = []
for x in base_rates:
    if x > 0.0: 
        screened_base_rates.append(str(x))

In [42]:
# Save times to a file
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/unconstrained_growth_rates.tsv', 'w') as rates:
    for x in screened_base_rates: rates.write(x + '\n')

### Contextualizing Published Datasets

In [21]:
# Read in transcriptomes

# Data collected from:
# Monk et al. (2016). Multi-omics Quantification of Species Variation of Escherichia coli
# Links Molecular Features with Strain Phenotypes. Cell Systems. 3; 238–251.

# Load in GPR translations
gpr_dict = {}
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/Monk_et_al_2016/iJO1366_genes.tsv', 'r') as genes:
    for line in genes:
        line = line.split()
        gpr_dict[line[1]] = line[0]

# Normalized abundances
# Separate into treatment goups and calculate medians
m9_aerobic = {}
m9_anaerobic = {}     
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/Monk_et_al_2016/normalized.tsv', 'r') as transcription:
    for line in transcription:
        line = line.split()
        if line[0] == 'gene':
            continue
        else:
            try:
                gene = gpr_dict[line[0]]
            except:
                continue
            m9_aerobic[gene] = numpy.median([int(x) for x in line[1:4]])
            m9_anaerobic[gene] = numpy.median([int(y) for y in line[4:7]])

# Rich media (LB) data from:
# Double-stranded transcriptome of E. coli
# Meghan Lybecker, Bob Zimmermann, Ivana Bilusic, Nadezda Tukhtubaeva, Renée Schroeder
# Proceedings of the National Academy of Sciences Feb 2014, 111 (8) 3134-3139; DOI: 10.1073/pnas.1315974111
lb_aerobic = {}     
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/Lybecker_2014.mapped.norm.tsv', 'r') as transcription:
    header = transcription.readline()
    for line in transcription: 
        line = line.split()
        lb_aerobic[line[0]] = float(line[1])

In [4]:
# Aerobic growth in M9 + glucose
iJO1366_m9_aerobic, m9_aerobic_samples = riptide(iJO1366, m9_aerobic)

Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 467 from 2583 (81.9% reduction)
Metabolites pruned to 466 from 1805 (74.2% reduction)

Flux through the objective REDUCED to 60.203 from 105.765 (43.08% shift)
Solution space ellipsoid volume DECREASED to ~0.262 from ~36.576 (99.28% shift)

RIPTiDe completed in 4.2 minutes


In [6]:
iJO1366_m9_aerobic

Name,iJO1366_riptide
Memory address,0x07f97fb6b0a90
Number of metabolites,466
Number of reactions,467
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


In [68]:
iJO1366_m9_aerobic.slim_optimize()

60.180610780793984

In [136]:
m9_aerobic_times = collect_doubling_times(m9_aerobic_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(m9_aerobic_times), numpy.median(m9_aerobic_times), max(m9_aerobic_times)])
m9_aerobic_rates = collect_growth_rates(m9_aerobic_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(m9_aerobic_rates), numpy.median(m9_aerobic_rates), max(m9_aerobic_rates)])

[65.36, 84.06, 86.4]
[0.694, 0.714, 0.918]


In [137]:
# Run in aerobic exchange conditions
#iJO1366_m9_anaerobic_test, m9_anaerobic_samples_test = riptide(iJO1366, m9_anaerobic)

In [138]:
#max_doubling_time(iJO1366_m9_anaerobic_test)

In [139]:
#m9_aerobic_rates_test = collect_growth_rates(m9_anaerobic_samples_test, 'BIOMASS_Ec_iJO1366_WT_53p95M')
#print([min(m9_aerobic_rates_test), numpy.median(m9_aerobic_rates_test), max(m9_aerobic_rates_test)])

In [5]:
# Anaerobic growth in M9 + glucose
#iJO1366.reactions.get_by_id('EX_o2_e').bounds = (0.0, 0.0) # make anaerobic
iJO1366_m9_anaerobic, m9_anaerobic_samples = riptide(iJO1366, m9_anaerobic)
#iJO1366.reactions.get_by_id('EX_o2_e').bounds = (-1000.0, 1000.0) # revert change

Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 501 from 2583 (80.6% reduction)
Metabolites pruned to 503 from 1805 (72.1% reduction)

Flux through the objective REDUCED to 41.662 from 105.765 (60.61% shift)
Solution space ellipsoid volume DECREASED to ~0.162 from ~36.576 (99.56% shift)

RIPTiDe completed in 5.6 minutes


In [8]:
iJO1366_m9_anaerobic

Name,iJO1366_riptide
Memory address,0x07f97f5348710
Number of metabolites,503
Number of reactions,501
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


In [ ]:
max_doubling_time(iJO1366_m9_anaerobic)

In [154]:
m9_anaerobic_times = collect_doubling_times(m9_anaerobic_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(m9_anaerobic_times), numpy.median(m9_anaerobic_times), max(m9_anaerobic_times)])
m9_anaerobic_rates = collect_growth_rates(m9_anaerobic_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(m9_anaerobic_rates), numpy.median(m9_anaerobic_rates), max(m9_anaerobic_rates)])

[90.55, 280.415, 401.4]
[0.149, 0.214, 0.663]


In [6]:
# Aerobic growth in LB
iJO1366_lb_aerobic, lb_samples = riptide(iJO1366, lb_aerobic)

Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 494 from 2583 (80.9% reduction)
Metabolites pruned to 495 from 1805 (72.6% reduction)

Flux through the objective REDUCED to 57.284 from 105.765 (45.84% shift)
Solution space ellipsoid volume DECREASED to ~0.354 from ~36.576 (99.03% shift)

RIPTiDe completed in 4.7 minutes


In [10]:
iJO1366_lb_aerobic

Name,iJO1366_riptide
Memory address,0x07f97e5f95e90
Number of metabolites,495
Number of reactions,494
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


In [30]:
max_doubling_time(iJO1366_lb_aerobic)

1.44 hours


In [153]:
lb_aerobic_times = collect_doubling_times(lb_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(lb_aerobic_times), numpy.median(lb_aerobic_times), max(lb_aerobic_times)])
lb_aerobic_rates = collect_growth_rates(lb_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(lb_aerobic_rates), numpy.median(lb_aerobic_rates), max(lb_aerobic_rates)])

[130.42, 358.015, 493.16]
[0.122, 0.168, 0.46]


In [22]:
# Compare to base implementation of pFBA
# All coefficients set to 1.0, so transcriptome is irrelevant
iJO1366_pfba, pfba_samples = riptide(iJO1366, m9_aerobic, coefficients=[1.0,1.0,1.0,1.0,1.0])

Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 449 from 2583 (82.6% reduction)
Metabolites pruned to 449 from 1805 (75.1% reduction)

Flux through the objective REDUCED to 56.29 from 105.765 (46.78% shift)
Solution space ellipsoid volume DECREASED to ~0.302 from ~9.161 (96.7% shift)

RIPTiDe completed in 3.8 minutes


In [14]:
iJO1366_pfba

Name,iJO1366_riptide
Memory address,0x07f7d70f15750
Number of metabolites,449
Number of reactions,449
Objective expression,0.0 + 1.0*BIOMASS_Ec_iJO1366_WT_53p95M - 1.0*BIOMASS_Ec_iJO1366_WT_53p95M_reverse_06c4a
Compartments,"periplasm, cytosol, extracellular space"


In [12]:
iJO1366_pfba.slim_optimize()

57.20042650657536

In [8]:
len(iJO1366_pfba.genes)

374

In [30]:
pfba_rates = collect_growth_rates(pfba_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(pfba_rates), numpy.median(pfba_rates), max(pfba_rates)])
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/pfba_growth_rates.tsv', 'w') as output_file:
    for x in pfba_rates: output_file.write(str(x) + '\n')

[0.694, 0.716, 0.938]


In [ ]:
write pfba_times to file
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/invivo_growth_rates.tsv', 'w') as output_file:
    for x in pfba_times: output_file.write(str(x) + '\n')

In [24]:
# Compares lists to create diagrams for 4 groups
def venn_comparison(list1, list2, list3, list4):
        
    # Confirm correct data types
    list1 = set(list1)
    list2 = set(list2)
    list3 = set(list3)
    list4 = set(list4)
    
    # Identify exclusive elements
    list1_only = list1.difference(list2)
    list1_only = list1_only.difference(list3)
    list1_only = list1_only.difference(list4)
    list2_only = list2.difference(list1)
    list2_only = list2_only.difference(list3)
    list2_only = list2_only.difference(list4)
    list3_only = list3.difference(list1)
    list3_only = list3_only.difference(list2)
    list3_only = list3_only.difference(list4)
    list4_only = list4.difference(list1)
    list4_only = list4_only.difference(list2)
    list4_only = list4_only.difference(list3)

    # Find overlap between just 2 groups
    list1_list2_overlap = list1.intersection(list2)
    list1_list2_overlap = list1_list2_overlap.difference(list3)
    list1_list2_overlap = list1_list2_overlap.difference(list4)
    list1_list3_overlap = list1.intersection(list3)
    list1_list3_overlap = list1_list3_overlap.difference(list2)
    list1_list3_overlap = list1_list3_overlap.difference(list4)
    list1_list4_overlap = list1.intersection(list4)
    list1_list4_overlap = list1_list4_overlap.difference(list2)
    list1_list4_overlap = list1_list4_overlap.difference(list3)
    list2_list3_overlap = list2.intersection(list3)
    list2_list3_overlap = list2_list3_overlap.difference(list1)
    list2_list3_overlap = list2_list3_overlap.difference(list4)
    list2_list4_overlap = list2.intersection(list4)
    list2_list4_overlap = list2_list4_overlap.difference(list1)
    list2_list4_overlap = list2_list4_overlap.difference(list3)
    list3_list4_overlap = list3.intersection(list4)
    list3_list4_overlap = list3_list4_overlap.difference(list1)
    list3_list4_overlap = list3_list4_overlap.difference(list2)

    # Find overlap in 3 groups
    list1_list2_list3_overlap = list1.intersection(list2)
    list1_list2_list3_overlap = list1_list2_list3_overlap.intersection(list3)
    list1_list2_list3_overlap = list1_list2_list3_overlap.difference(list4)
    list1_list2_list4_overlap = list1.intersection(list2)
    list1_list2_list4_overlap = list1_list2_list4_overlap.intersection(list4)
    list1_list2_list4_overlap = list1_list2_list4_overlap.difference(list3)
    list1_list3_list4_overlap = list1.intersection(list3)
    list1_list3_list4_overlap = list1_list3_list4_overlap.intersection(list4)
    list1_list3_list4_overlap = list1_list3_list4_overlap.difference(list2)
    list2_list3_list4_overlap = list2.intersection(list3)
    list2_list3_list4_overlap = list2_list3_list4_overlap.intersection(list4)
    list2_list3_list4_overlap = list2_list3_list4_overlap.difference(list1)
    
    # Find overlap between all groups
    all_list_overlap = list1.intersection(list2)
    all_list_overlap = all_list_overlap.intersection(list3)
    all_list_overlap = all_list_overlap.intersection(list4)
    
    # Calculate totals in each group
    list1_total = float(len(list1))
    list2_total = float(len(list2))
    list3_total = float(len(list3))
    list4_total = float(len(list4))
    list1_only_total = float(len(list1_only))
    list2_only_total = float(len(list2_only))
    list3_only_total = float(len(list3_only))
    list4_only_total = float(len(list4_only))
    list1_list2_overlap_total = float(len(list1_list2_overlap))
    list1_list3_overlap_total = float(len(list1_list3_overlap))
    list1_list4_overlap_total = float(len(list1_list4_overlap))
    list2_list3_overlap_total = float(len(list2_list3_overlap))
    list2_list4_overlap_total = float(len(list2_list4_overlap))
    list3_list4_overlap_total = float(len(list3_list4_overlap))
    list1_list2_list3_overlap_total = float(len(list1_list2_list3_overlap))
    list1_list2_list4_overlap_total = float(len(list1_list2_list4_overlap))
    list1_list3_list4_overlap_total = float(len(list1_list3_list4_overlap))
    list2_list3_list4_overlap_total = float(len(list2_list3_list4_overlap))
    all_list_overlap_total = float(len(all_list_overlap))
    
    # Calculate percent overlaps
    list1_only_percent = round(((list1_only_total / list1_total) * 100.0), 1)
    list2_only_percent = round(((list2_only_total / list2_total) * 100.0), 1)
    list3_only_percent = round(((list3_only_total / list3_total) * 100.0), 1)
    list4_only_percent = round(((list4_only_total / list4_total) * 100.0), 1)
    temp1 = (list1_list2_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_overlap_total / list2_total) * 100.0
    list1_list2_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_overlap_total / list3_total) * 100.0
    list1_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list4_overlap_total / list4_total) * 100.0
    list1_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list3_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_overlap_total / list3_total) * 100.0
    list2_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list4_overlap_total / list4_total) * 100.0
    list2_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list3_list4_overlap_total / list3_total) * 100.0
    temp2 = (list3_list4_overlap_total / list4_total) * 100.0
    list3_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list2_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list3_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list3_overlap_total / list3_total) * 100.0
    list1_list2_list3_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list2_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list4_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list4_overlap_total / list4_total) * 100.0
    list1_list2_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list3_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list1_list3_list4_overlap_total / list4_total) * 100.0
    list1_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list2_list3_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list2_list3_list4_overlap_total / list4_total) * 100.0
    list2_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (all_list_overlap_total / list1_total) * 100.0
    temp2 = (all_list_overlap_total / list2_total) * 100.0
    temp3 = (all_list_overlap_total / list3_total) * 100.0
    temp4 = (all_list_overlap_total / list4_total) * 100.0
    all_list_overlap_percent = round(numpy.mean([temp1, temp2, temp3, temp4]), 1)
    
    # Print report to the screen
    print('List 1 only: ' + str(list1_only_percent) + '% (' + str(int(list1_only_total)) + ')')
    print('List 2 only: ' + str(list2_only_percent) + '% (' + str(int(list2_only_total)) + ')')
    print('List 3 only: ' + str(list3_only_percent) + '% (' + str(int(list3_only_total)) + ')')
    print('List 4 only: ' + str(list4_only_percent) + '% (' + str(int(list4_only_total)) + ')')
    print('')
    print('List 1 + List 2: ' + str(list1_list2_overlap_percent) + '% (' + str(int(list1_list2_overlap_total)) + ')')
    print('List 1 + List 3: ' + str(list1_list3_overlap_percent) + '% (' + str(int(list1_list3_overlap_total)) + ')')
    print('List 1 + List 4: ' + str(list1_list4_overlap_percent) + '% (' + str(int(list1_list4_overlap_total)) + ')')
    print('List 2 + List 3: ' + str(list2_list3_overlap_percent) + '% (' + str(int(list2_list3_overlap_total)) + ')')
    print('List 2 + List 4: ' + str(list2_list4_overlap_percent) + '% (' + str(int(list2_list4_overlap_total)) + ')')
    print('List 3 + List 4: ' + str(list3_list4_overlap_percent) + '% (' + str(int(list3_list4_overlap_total)) + ')')
    print('')
    print('List 1 + List 2 + List 3: ' + str(list1_list2_list3_overlap_percent) + '% (' + str(int(list1_list2_list3_overlap_total)) + ')')
    print('List 1 + List 2 + List 4: ' + str(list1_list2_list4_overlap_percent) + '% (' + str(int(list1_list2_list4_overlap_total)) + ')')
    print('List 1 + List 3 + List 4: ' + str(list1_list3_list4_overlap_percent) + '% (' + str(int(list1_list3_list4_overlap_total)) + ')')
    print('List 2 + List 3 + List 4: ' + str(list2_list3_list4_overlap_percent) + '% (' + str(int(list2_list3_list4_overlap_total)) + ')')
    print('')
    print('Shared: ' + str(all_list_overlap_percent) + '% (' + str(int(all_list_overlap_total)) + ')')

    # Return new lists
    return [list1_only,list2_only,list3_only,list4_only,list1_list2_overlap, list1_list3_overlap, list1_list4_overlap, list2_list3_overlap, list2_list4_overlap, list3_list4_overlap, list1_list2_list3_overlap, list1_list2_list4_overlap, list1_list3_list4_overlap, list2_list3_list4_overlap, all_list_overlap]


In [33]:
# Reactions
iJO1366_m9_aerobic_reactions = [x.id for x in iJO1366_m9_aerobic.reactions]
iJO1366_m9_anaerobic_reactions = [x.id for x in iJO1366_m9_anaerobic.reactions]
iJO1366_lb_aerobic_reactions = [x.id for x in iJO1366_lb_aerobic.reactions]
iJO1366_pfba_reactions = [x.id for x in iJO1366_pfba.reactions]

reactions_comparisons = venn_comparison(iJO1366_pfba_reactions, iJO1366_lb_aerobic_reactions, iJO1366_m9_aerobic_reactions, iJO1366_m9_anaerobic_reactions)

# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

List 1 only: 3.8% (17)
List 2 only: 4.3% (21)
List 3 only: 4.1% (19)
List 4 only: 8.0% (40)

List 1 + List 2: 4.5% (21)
List 1 + List 3: 4.8% (22)
List 1 + List 4: 1.3% (6)
List 2 + List 3: 1.7% (8)
List 2 + List 4: 8.8% (44)
List 3 + List 4: 5.8% (28)

List 1 + List 2 + List 3: 4.7% (22)
List 1 + List 2 + List 4: 3.1% (15)
List 1 + List 3 + List 4: 1.1% (5)
List 2 + List 3 + List 4: 4.5% (22)

Shared: 71.5% (341)


In [34]:
# Metabolites
iJO1366_m9_aerobic_metabolites = [x.id for x in iJO1366_m9_aerobic.metabolites]
iJO1366_m9_anaerobic_metabolites = [x.id for x in iJO1366_m9_anaerobic.metabolites]
iJO1366_lb_aerobic_metabolites = [x.id for x in iJO1366_lb_aerobic.metabolites]
iJO1366_pfba_metabolites = [x.id for x in iJO1366_pfba.metabolites]

metabolites_comparisons = venn_comparison(iJO1366_pfba_metabolites, iJO1366_lb_aerobic_metabolites, iJO1366_m9_aerobic_metabolites, iJO1366_m9_anaerobic_metabolites)

# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

List 1 only: 1.3% (6)
List 2 only: 3.2% (16)
List 3 only: 1.9% (9)
List 4 only: 5.0% (25)

List 1 + List 2: 2.1% (10)
List 1 + List 3: 2.6% (12)
List 1 + List 4: 0.6% (3)
List 2 + List 3: 1.0% (5)
List 2 + List 4: 6.8% (34)
List 3 + List 4: 3.5% (17)

List 1 + List 2 + List 3: 1.7% (8)
List 1 + List 2 + List 4: 1.9% (9)
List 1 + List 3 + List 4: 0.4% (2)
List 2 + List 3 + List 4: 2.9% (14)

Shared: 83.6% (399)


In [35]:
# Screen context specific growth rates by each model's optimal growth

# Determine bounds
pfba_obj_val_lb = iJO1366_pfba.slim_optimize() * 0.8
lb_aerobic_obj_val_lb = iJO1366_lb_aerobic.slim_optimize() * 0.8
m9_aerobic_obj_val_lb = iJO1366_m9_aerobic.slim_optimize() * 0.8
m9_anaerobic_obj_val_lb = iJO1366_m9_anaerobic.slim_optimize() * 0.8
pfba_obj_val_ub = iJO1366_pfba.slim_optimize()
lb_aerobic_obj_val_ub = iJO1366_lb_aerobic.slim_optimize()
m9_aerobic_obj_val_ub = iJO1366_m9_aerobic.slim_optimize()
m9_anaerobic_obj_val_ub = iJO1366_m9_anaerobic.slim_optimize()

# Collect fluxes
pfba_biomass = list(pfba_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
lb_biomass = list(lb_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
m9_aerobic_biomass = list(m9_aerobic_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])
m9_anaerobic_biomass = list(m9_anaerobic_samples['BIOMASS_Ec_iJO1366_WT_53p95M'])

# Screen fluxes
pfba_biomass = [x for x in pfba_biomass if x >= pfba_obj_val_lb and x <= pfba_obj_val_ub]
lb_biomass = [x for x in lb_biomass if x >= lb_aerobic_obj_val_lb and x <= lb_aerobic_obj_val_ub]
m9_aerobic_biomass = [x for x in m9_aerobic_biomass if x >= m9_aerobic_obj_val_lb and x <= m9_aerobic_obj_val_ub]
m9_anaerobic_biomass = [x for x in m9_anaerobic_biomass if x >= m9_anaerobic_obj_val_lb and x <= m9_anaerobic_obj_val_ub]

# Convert to per hour rate
pfba_rates = [round((x / 60.0), 3) for x in pfba_biomass]
lb_aerobic_rates = [round((x / 60.0), 3) for x in lb_biomass]
m9_aerobic_rates = [round((x / 60.0), 3) for x in m9_aerobic_biomass]
m9_anaerobic_rates = [round((x / 60.0), 3) for x in m9_anaerobic_biomass]

# Subsample evenly
import random
sub_level = min([len(pfba_rates), len(lb_aerobic_rates), len(m9_aerobic_rates), len(m9_anaerobic_rates)])
pfba_sub = random.sample(range(0,len(pfba_rates)), sub_level)
lb_sub = random.sample(range(0,len(lb_aerobic_rates)), sub_level)
m9a_sub = random.sample(range(0,len(m9_aerobic_rates)), sub_level)
m9n_sub = random.sample(range(0,len(m9_anaerobic_rates)), sub_level)
pfba_rates = [pfba_rates[i] for i in pfba_sub]
lb_aerobic_rates = [lb_aerobic_rates[i] for i in lb_sub]
m9_aerobic_rates = [m9_aerobic_rates[i] for i in m9a_sub]
m9_anaerobic_rates = [m9_anaerobic_rates[i] for i in m9n_sub]

# Convert to strings
pfba_rates = [str(x) for x in pfba_rates]
pfba_rates = 'base_pfba\t' + '\t'.join(pfba_rates) + '\n'
m9_aerobic_rates = [str(x) for x in m9_aerobic_rates]
m9_aerobic_rates = 'm9_gluc_aerobic\t' + '\t'.join(m9_aerobic_rates) + '\n'
m9_anaerobic_rates = [str(x) for x in m9_anaerobic_rates]
m9_anaerobic_rates = 'm9_gluc_anaerobic\t' + '\t'.join(m9_anaerobic_rates) + '\n'
lb_aerobic_rates = [str(x) for x in lb_aerobic_rates]
lb_aerobic_rates = 'lb_aerobic\t' + '\t'.join(lb_aerobic_rates) + '\n'

# Write to file
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/new_growth_rates.tsv', 'w') as rates:
    rates.write(pfba_rates)
    rates.write(m9_aerobic_rates)
    rates.write(m9_anaerobic_rates)
    rates.write(lb_aerobic_rates)

In [19]:
# Write contextualized models to SBMLs and JSONs
cobra.io.write_sbml_model(iJO1366_m9_aerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.sbml')
cobra.io.save_json_model(iJO1366_m9_aerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.json')
cobra.io.write_sbml_model(iJO1366_m9_anaerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_anaerobic.sbml')
cobra.io.save_json_model(iJO1366_m9_anaerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_anaerobic.json')
cobra.io.write_sbml_model(iJO1366_lb_aerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.sbml')
cobra.io.save_json_model(iJO1366_lb_aerobic, '/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.json')

In [21]:
# Correct the sample labels
def label_flux_samples(file_name, label):
    new_name = file_name.rstrip('tsv') + 'format.tsv'
    new_file = open(new_name, 'w')
    
    with open(file_name, 'r') as samples:
        header = samples.readline()
        header = 'sample\t' + header
        new_file.write(header)
        current = 1
        
        for line in samples:
            line = label + '_' + str(current) + '\t' + line
            new_file.write(line)
            current += 1

    new_file.close()

In [27]:
# Write chosen flux sample tables to tsvs
m9_aerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_aerobic.flux_samples.tsv', sep='\t')
m9_anaerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_anaerobic.flux_samples.tsv', sep='\t')
lb_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/LB_aerobic.flux_samples.tsv', sep='\t')
pfba_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/pFBA.flux_samples.tsv', sep='\t')

    
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_aerobic.flux_samples.tsv', 'm9_aer')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/M9_anaerobic.flux_samples.tsv', 'm9_anaer')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/LB_aerobic.flux_samples.tsv', 'lb')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/pFBA.flux_samples.tsv', 'pfba')

### Context-specific Essentiality

In [8]:
import cobra.flux_analysis

In [19]:
iJO1366_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366)
print('Essential genes: ' + str(len(iJO1366_essential_genes)))

Essential genes: 105


In [20]:
iJO1366_pfba_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_pfba)
iJO1366_pfba_essential_genes = set([x.id for x in iJO1366_pfba_essential_genes])
print('Essential genes: ' + str(len(iJO1366_pfba_essential_genes)))

Essential genes: 226


In [21]:
iJO1366_lb_aerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_lb_aerobic)
iJO1366_lb_aerobic_essential_genes = set([x.id for x in iJO1366_lb_aerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_lb_aerobic_essential_genes)))

Essential genes: 227


In [22]:
iJO1366_m9_aerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_m9_aerobic)
iJO1366_m9_aerobic_essential_genes = set([x.id for x in iJO1366_m9_aerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_m9_aerobic_essential_genes)))

Essential genes: 232


In [23]:
iJO1366_m9_anaerobic_essential_genes = cobra.flux_analysis.find_essential_genes(iJO1366_m9_anaerobic)
iJO1366_m9_anaerobic_essential_genes = set([x.id for x in iJO1366_m9_anaerobic_essential_genes])
print('Essential genes: ' + str(len(iJO1366_m9_anaerobic_essential_genes)))

Essential genes: 229


In [14]:
essentiality_comparisons = venn_comparison(iJO1366_pfba_essential_genes, iJO1366_lb_aerobic_essential_genes, iJO1366_m9_aerobic_essential_genes, iJO1366_m9_anaerobic_essential_genes)
# List 1: pfba
# List 2: lb_aerobic
# List 3: m9_aerobic
# List 4: m9_anaerobic

NameError: name 'venn_comparison' is not defined

In [15]:
# Find those genes shared in all models
core_essential = iJO1366_pfba_essential_genes.intersection(iJO1366_lb_aerobic_essential_genes)
core_essential = core_essential.intersection(iJO1366_m9_aerobic_essential_genes)
core_essential = core_essential.intersection(iJO1366_m9_anaerobic_essential_genes)
print('Essential in all GENREs: ' + str(len(core_essential)))

# Substract as background from each
iJO1366_pfba_essential_genes = iJO1366_pfba_essential_genes.difference(core_essential)
iJO1366_lb_aerobic_essential_genes = iJO1366_lb_aerobic_essential_genes.difference(core_essential)
iJO1366_m9_aerobic_essential_genes = iJO1366_m9_aerobic_essential_genes.difference(core_essential)
iJO1366_m9_anaerobic_essential_genes = iJO1366_m9_anaerobic_essential_genes.difference(core_essential)

# Find non-essentiality across models
iJO1366_pfba_genes = set([x.id for x in iJO1366_pfba.genes])
iJO1366_lb_aerobic_genes = set([x.id for x in iJO1366_lb_aerobic.genes])
iJO1366_m9_aerobic_genes = set([x.id for x in iJO1366_m9_aerobic.genes])
iJO1366_m9_anaerobic_genes = set([x.id for x in iJO1366_m9_anaerobic.genes])

# Compare overlapping genes
total_genes = set()
total_genes |= iJO1366_pfba_essential_genes
total_genes |= iJO1366_lb_aerobic_essential_genes
total_genes |= iJO1366_m9_aerobic_essential_genes
total_genes |= iJO1366_m9_anaerobic_essential_genes
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/essentiality_test.tsv', 'w') as outfile:
    outfile.write('gene\tpfba\tlb_aerobic\tm9_aerobic\tm9_anaerobic\n')
    for gene in total_genes:
        entry = ['filler','filler','filler','filler']
        
        if gene in iJO1366_pfba_essential_genes:
            entry[0] = 2
        elif gene in iJO1366_pfba_genes:
            entry[0] = 1
        else:
            entry[0] = 0
            
        if gene in iJO1366_lb_aerobic_essential_genes:
            entry[1] = 2
        elif gene in iJO1366_lb_aerobic_genes:
            entry[1] = 1
        else:
            entry[1] = 0
            
        if gene in iJO1366_m9_aerobic_essential_genes:
            entry[2] = 2
        elif gene in iJO1366_m9_aerobic_genes:
            entry[2] = 1
        else:
            entry[2] = 0
            
        if gene in iJO1366_m9_anaerobic_essential_genes:
            entry[3] = 2
        elif gene in iJO1366_m9_anaerobic_genes:
            entry[3] = 1
        else:
            entry[3] = 0
            
        entry = gene + '\t' + '\t'.join([str(x) for x in entry]) + '\n'
        outfile.write(entry)


Essential in all GENREs: 178


### Metatranscriptomic analysis

In [69]:
def find_source(model, met_id):
    generating = set()
    for rxn in model.reactions:
        for met in rxn.products:
            if met_id in met.id:
                generating |= set([rxn.id])
        
    print('Metabolite sources: ' + str(len(generating)))
    return generating

In [18]:
clinda_k12_metaT = {}
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/transcript/clinda_k12.mapped.norm.tsv', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        clinda_k12_metaT[line[0]] = float(line[2])

In [19]:
iJO1366_invivo_metaT, invivo_metaT_samples = riptide(iJO1366, clinda_k12_metaT)

Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific solution space (longest step)...

Reactions pruned to 490 from 2583 (81.0% reduction)
Metabolites pruned to 493 from 1805 (72.7% reduction)

Flux through the objective REDUCED to 53.17 from 105.765 (49.73% shift)
Solution space ellipsoid volume DECREASED to ~0.306 from ~9.161 (96.66% shift)

RIPTiDe completed in 4.9 minutes


In [29]:
invivo_rates = collect_growth_rates(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_rates), numpy.median(invivo_rates), max(invivo_rates)])
with open('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/invivo_growth_rates.tsv', 'w') as output_file:
    for x in invivo_rates: output_file.write(str(x) + '\n')

[0.538, 0.578, 0.886]


In [36]:
invivo_anaerobic_atp = find_source(iJO1366_invivo_metaT, 'atp_c')

Metabolite sources: 3


In [54]:
vitro_ex = set([x.id for x in iJO1366_m9_anaerobic.reactions if 'EX_' in x.id])
vivo_ex = set([x.id for x in iJO1366_invivo_metaT.reactions if 'EX_' in x.id])

vitro_ex_only = vitro_ex.difference(vivo_ex)
vitro_ex_only_input = set()
for y in vitro_ex_only:
    if abs(iJO1366_m9_anaerobic.reactions.get_by_id(y).lower_bound) > abs(iJO1366_m9_anaerobic.reactions.get_by_id(y).upper_bound):
        vitro_ex_only_input |= set([y])

vivo_ex_only = vivo_ex.difference(vitro_ex)
vivo_ex_only_input = set()
for y in vivo_ex_only:
    if abs(iJO1366_invivo_metaT.reactions.get_by_id(y).lower_bound) > abs(iJO1366_invivo_metaT.reactions.get_by_id(y).upper_bound):
        vivo_ex_only_input |= set([y])

In [58]:
for x in vitro_ex_only_input: print(iJO1366_m9_anaerobic.reactions.get_by_id(x).reactants[0].name)

Quinate
Glycerol 3-phosphate
Glycerophosphoserine
L-Leucine
L-tartrate
Ethanol
Guanosine
D-Fructose 6-phosphate
Allantoin


In [59]:
for x in vivo_ex_only_input: print(iJO1366_invivo_metaT.reactions.get_by_id(x).reactants[0].name)

L-Serine
Deoxyuridine
Trehalose
Deoxyguanosine
Glycine
Pyridoxine
Myo-Inositol hexakisphosphate
Shikimate


In [19]:
max_doubling_time(iJO1366_invivo_metaT)

1.11 hours


In [22]:
# Write flux sample tables to tsv
m9_anaerobic_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invitro.flux_samples.tsv', sep='\t')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invitro.flux_samples.tsv', 'invitro')

invivo_metaT_samples.to_csv('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invivo.flux_samples.tsv', sep='\t')
label_flux_samples('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/flux_samples/invivo.flux_samples.tsv', 'invivo')

In [17]:
invivo_metaT_times = collect_doubling_times(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_metaT_times), numpy.median(invivo_metaT_times), max(invivo_metaT_times)])
invivo_metaT_rates = collect_growth_rates(invivo_metaT_samples, 'BIOMASS_Ec_iJO1366_WT_53p95M')
print([min(invivo_metaT_rates), numpy.median(invivo_metaT_rates), max(invivo_metaT_rates)])

[86.41, 105.13, 111.6]
[0.538, 0.571, 0.694]


In [70]:
m9 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_m9_aerobic.sbml')
lb = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/riptide_models/iJO1366_lb_aerobic.sbml')

In [71]:
m9_nadph = find_source(m9, 'nadph_c')
lb_nadph = find_source(lb, 'nadph_c')

Metabolite sources: 4
Metabolite sources: 3


### Testing Previous Integration Algorithms

In [ ]:
iJO1366 = cobra.io.read_sbml_model('/home/mjenior/Desktop/repos/Jenior_RIPTiDe_2019/data/iJO1366.xml')
iJO1366.objective = iJO1366.reactions.BIOMASS_Ec_iJO1366_WT_53p95M

# Open all exchanges
exchanges = set()
for rxn in iJO1366.reactions:
    if len(rxn.reactants) == 0 or len(rxn.products) == 0:
        rxn.bounds = (min(rxn.lower_bound, -1000), max(rxn.upper_bound, 1000))
        exchanges |= set([rxn.id])

In [4]:
# Comparison to GIMME and iMAT
import copy
import cobra
from driven.flux_analysis.transcriptomics import *

# Read in formatted data
m9_aerobic_driven = ExpressionProfile.from_csv('/home/mjenior/Desktop/m9_aerobic_expression.csv')

/home/mjenior/anaconda2/lib/python2.7/site-packages/driven-0.0.3-py2.7.egg/driven/data_sets/expression_profile.py:67: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



In [5]:
# iMAT
start_time = time.time()

iJO1366_imat_result = imat(iJO1366, expression_profile=m9_aerobic_driven, low_cutoff=102, high_cutoff=822)

duration = time.time() - start_time
duration = round(duration)

print('iMAT finished in ' + str(duration) + ' seconds')

iMAT finished in 44.0 seconds


In [35]:
# Check for overlap with lowly expressed reactions and gapfilled
from cobra.flux_analysis.parsimonious import *

imat_pfba = pfba(iJO1366, fraction_of_optimum=0.8)
test1 = imat_pfba.fluxes[imat_pfba.fluxes > 0.0]
test1 = set(test1.index)

test2 = set()
for index in iJO1366_imat_result.lowly_express.keys():
    if iJO1366_imat_result.lowly_express[index] == True:
        test2 |= set([index])

gapfilled_imat = test1.intersection(test2)
print(len(gapfilled_imat))

97


In [30]:
gapfilled_imat

{'3PEPTabcpp',
 'ADCL',
 'ADCS',
 'ADOCBLabcpp',
 'ADOCBLtonex',
 'AGPAT160',
 'AGPAT161',
 'AGPAT181',
 'ALLTN',
 'ALLTNt2rpp',
 'ARGORNt7pp',
 'ASNt2rpp',
 'BMOGDS1',
 'BMOGDS2',
 'CLt3_2pp',
 'CPMPS',
 'CPPPGO',
 'CU2tpp',
 'DHFR',
 'DHNAOT4',
 'DHNPA2r',
 'DPCOAK',
 'DRPA',
 'DTMPK',
 'FACOAE160',
 'FACOAE161',
 'FACOAE181',
 'FACOAL160t2pp',
 'FACOAL161t2pp',
 'FACOAL180t2pp',
 'FACOAL181t2pp',
 'FCLT',
 'FE2tpp',
 'FEENTERabcpp',
 'FESR',
 'FLDR2',
 'G3PAT160',
 'G3PAT161',
 'G3PAT181',
 'G6Pt6_2pp',
 'GAM6Pt6_2pp',
 'GLUt2rpp',
 'GLUt4pp',
 'GLYC3Pt6pp',
 'GLYCDx',
 'GSNt2pp',
 'GTHRDabcpp',
 'HPPK2',
 'I2FE2SS',
 'I2FE2SS2',
 'ILEt2rpp',
 'INDOLEt2rpp',
 'LEUt2rpp',
 'LIPAMPL',
 'LIPATPT',
 'MECDPS',
 'MEPCT',
 'METabcpp',
 'MN2tpp',
 'MOADSUx',
 'MOCDS',
 'MOGDS',
 'MPTAT',
 'MPTG',
 'MPTG2',
 'MPTS',
 'MPTSS',
 'MTHFR2',
 'NADH16pp',
 'NADH17pp',
 'NADK',
 'NO2t2rpp',
 'NO3R1pp',
 'NO3R2pp',
 'NO3t7pp',
 'NTRIR2x',
 'OPHBDC',
 'PFL',
 'PGPP160',
 'PGPP161',
 'PGPP181',
 'PGSA

In [32]:
# GIMME
start_time = time.time()

iJO1366_gimme_result = gimme(iJO1366, cutoff=102, expression_profile=m9_aerobic_driven)

duration = time.time() - start_time
duration = round(duration)

print('GIMME finished in ' + str(duration) + ' seconds')

GIMME finished in 33.0 seconds


In [33]:
# Integrate GIMME results
current = 0
remove = []
for entry in iJO1366_gimme_result.data_frame['inconsistency_scores']:
    if entry != 0.0:
        remove.append(list(iJO1366_gimme_result.fluxes.index)[current])
    current += 1

iJO1366_gimme = copy.deepcopy(iJO1366)

# Constrain fluxes to match output
for rxn_id, flux in iJO1366_gimme_result.fluxes.items():
    iJO1366_gimme.reactions.get_by_id(rxn_id).bounds = (flux, flux)
    
# Prune inactive reactions
for rxn in remove:
    iJO1366_gimme.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)
iJO1366_gimme.reactions.get_by_id('DM_4crsol_c').remove_from_model(remove_orphans=True)
print(len(remove))
removed = 1
while removed == 1:
    removed = 0
    for cpd in iJO1366_gimme.metabolites:
        if len(cpd.reactions) == 0:
            cpd.remove_from_model(); removed = 1
    for rxn in iJO1366_gimme.reactions:
        if len(rxn.metabolites) == 0: 
            rxn.remove_from_model(); removed = 1

# Test for growth
print('Objective value: ' + str(iJO1366_gimme.slim_optimize()))

130
Objective value: nan


In [34]:
remove

['ABUTt2pp',
 'ACACT2r',
 'ACACT3r',
 'ACACT4r',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'ACACT8r',
 'ADCL',
 'ADCS',
 'ADOCBLabcpp',
 'AGPAT160',
 'AGPAT161',
 'AGPAT181',
 'ALAGLUE',
 'ALLTN',
 'ALLTNt2rpp',
 'ARGORNt7pp',
 'ASNt2rpp',
 'BMOGDS1',
 'BMOGDS2',
 'CAt6pp',
 'CLt3_2pp',
 'CPMPS',
 'CPPPGO',
 'CTECOAI7',
 'CTECOAI8',
 'CU2tpp',
 'D_LACt2pp',
 'DHFR',
 'DHNAOT4',
 'DHNPA2r',
 'DPCOAK',
 'DRPA',
 'DTMPK',
 'ECOAH1',
 'ECOAH2',
 'ECOAH3',
 'ECOAH4',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'FACOAE160',
 'FACOAE161',
 'FACOAE181',
 'FCLT',
 'FE2tpp',
 'FEENTERabcpp',
 'FESR',
 'FLDR2',
 'GAM6Pt6_2pp',
 'GLYAT',
 'GLYC3Pt6pp',
 'GLYCLTt2rpp',
 'GSNt2pp',
 'GTHRDabcpp',
 'HACD1',
 'HACD2',
 'HACD3',
 'HACD4',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'HPPK2',
 'HYD1pp',
 'I2FE2SS',
 'I2FE2SS2',
 'ILEt2rpp',
 'INDOLEt2rpp',
 'IPDDI',
 'LALDO2x',
 'LEUt2rpp',
 'LIPAMPL',
 'LIPATPT',
 'MAN6Pt6_2pp',
 'MECDPS',
 'MEPCT',
 'METabcpp',
 'MGSA',
 'MN2tpp',
 'MOADSUx',
 'MOCDS',
 'MOGD

In [40]:
# Additional test datasets
# Gao, Y., Yurkovich, J. T., Seo, S. W., Kabimoldayev, I., Dräger, A., Chen, K., … Palsson, B. O. (2018). 
# Systematic discovery of uncharacterized transcription factors in Escherichia coli K-12 MG1655. 
# Nucleic Acids Research. https://doi.org/10.1093/nar/gky752

mops_glc = {}
with open('/home/mjenior/Desktop/Gao_et_al_2018/GSM3022135_wt_glc1.txt', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        if len(line) < 2: continue
        mops_glc[line[0]] = float(line[1])

wt_ph5 = {}
with open('/home/mjenior/Desktop/Gao_et_al_2018/GSM3108934_wt_ph5_1.txt', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        if len(line) < 2: continue
        wt_ph5[line[0]] = float(line[1])

wt_ph8 = {}
with open('/home/mjenior/Desktop/Gao_et_al_2018/GSM3108936_wt_ph8_1.txt', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        if len(line) < 2: continue
        wt_ph8[line[0]] = float(line[1])

ydcI_ph5 = {}
with open('/home/mjenior/Desktop/Gao_et_al_2018/GSM3108944_delydci_ph5_1.txt', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        if len(line) < 2: continue
        ydcI_ph5[line[0]] = float(line[1])

ydcI_ph8 = {}
with open('/home/mjenior/Desktop/Gao_et_al_2018/GSM3108946_delydci_ph8_1.txt', 'r') as transcription:
    header = transcription.readline()
    for line in transcription:
        line = line.split()
        if len(line) < 2: continue
        ydcI_ph8[line[0]] = float(line[1])


### Analyzing the C. difficile 630 model

In [2]:
# Read in transcript abundance table
def read_transcription_all(transcript_table):
    
    cef = {}
    clinda = {}
    strep = {}
    gnoto = {}
    
    with open(transcript_table, 'r') as transcripts:
        firstline = transcripts.readline()
        
        for line in transcripts:
            line = line.split(',')
            
            cef[str(line[0])] = float(line[1])
            clinda[str(line[0])] = float(line[2])
            strep[str(line[0])] = float(line[3])
            gnoto[str(line[0])] = float(line[4])
        
    return cef, clinda, strep, gnoto


In [10]:
# Read in all transcription 
cef_dict, clinda_dict, strep_dict, gnoto_dict = read_transcription_all('data/transcript/cdf_transcription.sub.format.csv')

In [49]:
iCdJ794 = cobra.io.read_sbml_model('data/iCdJ794.sbml')

In [7]:
# Substrate demand calculations

import numpy
import pandas
import operator
from cobra.flux_analysis.parsimonious import *
from cobra.flux_analysis.geometric import *

# Calculate extracellular metabolite shadow prices
def sampled_exchange_fluxes(model, flux_samples):
    
    exch_fluxes = {}
    for rxn in model.reactions: 
        if 'EX_' not in rxn.id: continue
        exch_fluxes[rxn.id] = []
    
    print('Collecting exchange fluxes...')
    with model as m:
        for index in range(0, len(flux_samples.index)):
            for rxn in flux_samples.columns:
                try:
                    m.reactions.get_by_id(rxn).bounds = (list(flux_samples[rxn])[index], list(flux_samples[rxn])[index])
                except:
                    continue
                    
            solution = m.optimize()
            for rxn, flux in solution.fluxes.iteritems():
                if 'EX_' not in rxn: continue
                exch_fluxes[rxn].append(flux)

    print('Calculating ranges...')
    summary_stats = {}
    for rxn in exch_fluxes.keys():
        substrate = model.reactions.get_by_any(rxn).metabolites[0].name
        summary_stats[rxn] = [substrate, numpy.percentile(exch_fluxes[rxn], 25), numpy.median(exch_fluxes[rxn]), numpy.percentile(exch_fluxes[rxn], 75),]

    ranking = []
    for rxn in summary_stats.keys(): ranking.append([rxn] + summary_stats[rxn])
    ranking = sorted(ranking, key=operator.itemgetter(3), reverse=True)
    temp_dict = {}
    for x in ranking: temp_dict[x[0]] = x[1:]
    flux_df = pandas.DataFrame.from_dict(temp_dict, orient='index', columns=['Name', 'Q25', 'Median', 'Q75'])
    
    return flux_df


### Cross-reference shadow price with metabolomics data

In [5]:
# Metabolomics

# Read in metabolomics data and collect intensities groups
def read_metabolomics(intensities_file, group1, group2):
    
    with open(intensities_file, 'r') as intensities:
        
        header = intensities.readline().split()
        group1_idx = []
        for index in group1:
            group1_idx.append(group1.index(index))
        group2_idx = []
        for index in group2:
            group2_idx.append(group2.index(index))
        
        group1_dict = {}
        group2_dict = {}
        for line in intensities:
            group1_dict[line[0]] = [float(line[x]) for x in group1_idx]
            group2_dict[line[0]] = [float(line[x]) for x in group2_idx]
             
    return group1_dict, group2_dict


# Test direction of change and significant differences in metaboalite values
def test_differences(dict1, dict2, cutoff=0.05):
    
    diff_dict = {}
    for index in dict1.keys():
        
        median1 = median(dict1[index])
        median2 = median(dict2[index])
        if median1 > median2:
            direction = 1
        elif median1 < median2:
            direction = -1
        else:
            direction = 0
            
        pval = round(list(scipy.stats.wilcoxon(x=dict1[index], y=dict2[index], zero_method='wilcox'))[1], 3)
        if pval <= cutoff:
            continue
        else:    
            diff_dict[index] = [direction, sig]
    
    return diff_dict


# Identify required grwoth substrates
def identify_requirements(model):
    
    with model as m:
        solution = flux_variability_analysis(m)
        
        necessary = []
        for index, row in solution.iterrows():
            if 'EX_' not in index:
                continue
            elif row['minimum'] < 0.0 and row['maximum'] <= 0.0:
                cpd = list(m.reactions.get_by_id(index).metabolites)[0].id
                necessary.append(cpd)
    
    return(necessary)


# Alter exchange fluxes based on metabolomic shifts
def integrate_changes(model, shifts, required):
    
    new_model = copy.deepcopy(model)
    for rxn in new_model.reactions:
        if 'EX_' in rxn.id:
            
            cpd = list(rxn.metabolites)[0].id
            if cpd in required:
                continue
            else:
                
                #integrate data here, just remove the reaction i guess...
                if shifts[rxn.id][0] 
                    rxn.remove_from_model()
                
                
        else:
            continue
                      
    return new_model 

    
    
    

SyntaxError: invalid syntax (<ipython-input-5-d8f8d915704f>, line 79)

In [109]:
# Read in metabolomic results
untreated_mock = {}
cef_630 = {}
cef_mock = {}
clinda_630 = {}
clinda_mock = {}
strep_630 = {}
strep_mock = {}
gnoto_630 = {}
gnoto_mock = {}

with open('data/metabolome/scaled_intensities.tsv', 'r') as metabolome:
    
    firstLine = metabolome.readline()
    
    for line in metabolome:
        line = line.split()
        
        untreated_mock[line[0]] = numpy.median([float(x) for x in line[5:14]])
        cef_630[line[0]] = numpy.median([float(x) for x in line[14:23]])
        cef_mock[line[0]] = numpy.median([float(x) for x in line[23:32]])
        clinda_630[line[0]] = numpy.median([float(x) for x in line[32:41]])
        clinda_mock[line[0]] = numpy.median([float(x) for x in line[41:50]])
        strep_630[line[0]] = numpy.median([float(x) for x in line[50:59]])
        strep_mock[line[0]] = numpy.median([float(x) for x in line[59:68]])
        gnoto_630[line[0]] = numpy.median([float(x) for x in line[68:77]])
        gnoto_mock[line[0]] = numpy.median([float(x) for x in line[77:86]])


In [135]:
# calculates change in concentration of a metabolite across metabolomes
def compare_concentration(metabolome1, metabolome2, metabolite):
    
    conc1 = 10 ** metabolome1[metabolite]
    conc2 = 10 ** metabolome2[metabolite]
    change = conc2 - conc1
    
    if change == 0.0:
        change = change
    elif change < 0.0:
        change = -numpy.log10(abs(change))
    else:
        change = numpy.log10(change)

    print(metabolite + ': ' + str(change))

In [139]:
# Cefoperazone
compare_concentration(cef_mock, cef_630, 'fructose')
compare_concentration(cef_mock, cef_630, 'N-acetyl-beta-glucosaminylamine')

fructose: 1.2869729381320378
N-acetyl-beta-glucosaminylamine: -0.3669198398437978


In [140]:
# Clindamycin
compare_concentration(clinda_mock, clinda_630, 'fructose')

fructose: 2.0225923028374746


In [141]:
# Streptomycin
compare_concentration(strep_mock, strep_630, 'fructose')

fructose: 1.411164101349768


In [142]:
# Gnotobiotic
compare_concentration(strep_mock, strep_630, 'fructose')
compare_concentration(strep_mock, strep_630, 'proline')

fructose: 1.411164101349768
proline: -2.2705935392452883
